In [5]:
paths_test = [ '../data-test/airlines.json', 
            '../data-test/gists.json', 
            '../data-test/historical-events.json',#have to rename the same value in json (movie on  the same level)
            '../data-test/movies.json',
            '../data-test/reddit.json',
            '../data-test/nasa.json']
path=paths_test[3]

In [26]:
import pandas as pd
import pyarrow as pa
import json

def make_new_json(lists,table_name):
    print(table_name)
    data = []
    for i in range(len(lists)):
        for item in lists[i]:
            item_data={
                "row_number": i,
                "value": item
            }
            data.append(item_data)
    json_object = json.dumps(data)
    return json_object

def delete_lists(df):
    is_list = df.apply(lambda x: any(isinstance(i, list) for i in x))

    list_columns = is_list[is_list].index.tolist()
    if(len(list_columns)>0):
        list_df = df[list_columns].copy()

        main_df = df.drop(list_columns, axis=1)
        main_df['row_number'] = range(1, len(main_df) + 1)
    main_table = pa.Table.from_pandas(main_df)
    print(main_table)
    if(len(list_columns)>0):
        for dropped_list in list_columns:
            new_json=make_new_json(list_df[dropped_list],dropped_list)
            print(new_json)
            # delete_lists(new_json)

with open(path, encoding='utf-8') as f:
    dt = json.load(f)

df = pd.json_normalize(dt)
delete_lists(df)



pyarrow.Table
title: string
year: int64
href: string
extract: string
thumbnail: string
thumbnail_width: double
thumbnail_height: double
row_number: int64
----
title: [["After Dark in Central Park","Clowns Spinning Hats","Capture of Boer Battery by British"]]
year: [[1900,1900,1900]]
href: [[null,"Clowns_Spinning_Hats","Capture_of_Boer_Battery_by_British"]]
extract: [[null,"Clowns Spinning Hats is a black-and-white silent film featuring clowns throwing hats back and forth to each other. It was written and produced by Lubin Films and released April 7, 1900.","Capture of Boer Battery by British is a black-and-white silent short docu-fiction film produced by James H. White for Edison Manufacturing Company in 1900. It is one minute in length and depicts the resistance of the Gordon Highlanders to the oncoming fire of the Boer's advance during the Boer War. It was filmed in West Orange, New Jersey USA and released April 14, 1900."]]
thumbnail: [[null,null,"https://upload.wikimedia.org/wikipe

In [6]:
import pandas as pd
import pyarrow as pa
import json

def make_new_json(lists,table_name):
    print(table_name)
    data = []
    for i in range(len(lists)):
        for item in lists[i]:
            item_data={
                "row_number": i,
                "value": item
            }
            data.append(item_data)
    return data

def delete_lists(df,name):
    
    print("#"*24)
    print("#"*((24-len(name))//2)+name+"#"*((24-len(name))//2))
    print("#"*24)

    df=pd.json_normalize(df)

    is_list = df.apply(lambda x: any(isinstance(i, list) for i in x))

    list_columns = is_list[is_list].index.tolist()
    if(len(list_columns)>0):
        list_df = df[list_columns].copy()

        df = df.drop(list_columns, axis=1)
        df['row_number'] = range(1, len(df) + 1)
    main_table = pa.Table.from_pandas(df)
    print(main_table)
    if(len(list_columns)>0):
        for dropped_list in list_columns:
            new_json=make_new_json(list_df[dropped_list],dropped_list)
            delete_lists(new_json,dropped_list)

with open(path, encoding='utf-8') as f:
    dt = json.load(f)

delete_lists(dt,'main')



########################
##########main##########
########################
pyarrow.Table
title: string
year: int64
href: string
extract: string
thumbnail: string
thumbnail_width: double
thumbnail_height: double
row_number: int64
----
title: [["After Dark in Central Park","Clowns Spinning Hats","Capture of Boer Battery by British"]]
year: [[1900,1900,1900]]
href: [[null,"Clowns_Spinning_Hats","Capture_of_Boer_Battery_by_British"]]
extract: [[null,"Clowns Spinning Hats is a black-and-white silent film featuring clowns throwing hats back and forth to each other. It was written and produced by Lubin Films and released April 7, 1900.","Capture of Boer Battery by British is a black-and-white silent short docu-fiction film produced by James H. White for Edison Manufacturing Company in 1900. It is one minute in length and depicts the resistance of the Gordon Highlanders to the oncoming fire of the Boer's advance during the Boer War. It was filmed in West Orange, New Jersey USA and released Apr

In [14]:
import pandas as pd
import pyarrow as pa
import json

def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                attribute_name= name + a +'.'
                flatten(x[a], attribute_name)
        elif type(x) is list:
            i = 0
            for a in x:
                attribute_name=name.rstrip('.') + '[' + str(i) + '].'
                flatten(a, attribute_name)
                i += 1
        else:
            out[name.rstrip('.')] = x

    flatten(y)
    return out

def json_to_table(json_data):
    # Spłaszczanie JSON
    flat_json = flatten_json(json_data)

    # Konwersja do DataFrame
    df = pd.DataFrame([flat_json])

    # Dodanie numeru wiersza
    df['row_number'] = df.index + 1

    # Konwersja do PyArrow Table
    table = pa.Table.from_pandas(df)

    return table


In [16]:
with open(paths_test[3], encoding='utf-8') as f:
    dt = json.load(f)
json_to_table(dt)

pyarrow.Table
[0].title: string
[0].year: int64
[0].href: null
[1].title: string
[1].year: int64
[1].genres[0]: string
[1].href: string
[1].extract: string
[2].title: string
[2].year: int64
[2].genres[0]: string
[2].genres[1]: string
[2].genres[2]: string
[2].href: string
[2].extract: string
[2].thumbnail: string
[2].thumbnail_width: int64
[2].thumbnail_height: int64
row_number: int64
----
[0].title: [["After Dark in Central Park"]]
[0].year: [[1900]]
[0].href: [1 nulls]
[1].title: [["Clowns Spinning Hats"]]
[1].year: [[1900]]
[1].genres[0]: [["Silent"]]
[1].href: [["Clowns_Spinning_Hats"]]
[1].extract: [["Clowns Spinning Hats is a black-and-white silent film featuring clowns throwing hats back and forth to each other. It was written and produced by Lubin Films and released April 7, 1900."]]
[2].title: [["Capture of Boer Battery by British"]]
[2].year: [[1900]]
...

In [22]:
import pandas as pd
import pyarrow as pa
import json

def flatten_json(y):
    out = {}
    tables = []

    def flatten(x, name=''):
        nonlocal tables
        if type(x) is dict:
            for a in x:
                attribute_name= name + a +'.'
                flatten(x[a], attribute_name)
        elif type(x) is list:
            i = 0
            for a in x:
                attribute_name=name.rstrip('.') + '[' + str(i) + '].'
                if type(a) is dict or type(a) is list:
                    flatten(a, attribute_name)
                else:
                    out[attribute_name.rstrip('.')] = a
                i += 1
            # Create a new table for the list
            list_df = pd.DataFrame(x)
            list_df['row_number'] = name.split('.')[0]
            list_table = pa.Table.from_pandas(list_df)
            tables.append((name.rstrip('.'), list_table))
        else:
            out[name.rstrip('.')] = x

    flatten(y)
    return out, tables

def json_to_tables(json_data):
    # Spłaszczanie JSON
    flat_json, list_tables = flatten_json(json_data)

    # Konwersja do DataFrame
    df = pd.DataFrame([flat_json])

    # Dodanie numeru wiersza
    df['row_number'] = df.index + 1

    # Konwersja do PyArrow Table
    main_table = pa.Table.from_pandas(df)

    return main_table, list_tables


In [23]:
with open(paths_test[3], encoding='utf-8') as f:
    dt = json.load(f)
json_to_tables(dt)

C:\Users\tusiu\AppData\Local\Temp\ipykernel_17048\192641810.py:27: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  list_table = pa.Table.from_pandas(list_df)
C:\Users\tusiu\AppData\Local\Temp\ipykernel_17048\192641810.py:27: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  list_table = pa.Table.from_pandas(list_df)


(pyarrow.Table
 [0].title: string
 [0].year: int64
 [0].href: null
 [1].title: string
 [1].year: int64
 [1].genres[0]: string
 [1].href: string
 [1].extract: string
 [2].title: string
 [2].year: int64
 [2].genres[0]: string
 [2].genres[1]: string
 [2].genres[2]: string
 [2].href: string
 [2].extract: string
 [2].thumbnail: string
 [2].thumbnail_width: int64
 [2].thumbnail_height: int64
 row_number: int64
 ----
 [0].title: [["After Dark in Central Park"]]
 [0].year: [[1900]]
 [0].href: [1 nulls]
 [1].title: [["Clowns Spinning Hats"]]
 [1].year: [[1900]]
 [1].genres[0]: [["Silent"]]
 [1].href: [["Clowns_Spinning_Hats"]]
 [1].extract: [["Clowns Spinning Hats is a black-and-white silent film featuring clowns throwing hats back and forth to each other. It was written and produced by Lubin Films and released April 7, 1900."]]
 [2].title: [["Capture of Boer Battery by British"]]
 [2].year: [[1900]]
 ...,
 [('[0].cast',
   pyarrow.Table
   row_number: null
   __index_level_0__: null
   ----
  

In [24]:
import pandas as pd
import pyarrow as pa
import json

def flatten_json(y):
    out = {}
    tables = []

    def flatten(x, name=''):
        nonlocal tables
        if type(x) is dict:
            for a in x:
                attribute_name= name + a +'.'
                flatten(x[a], attribute_name)
        elif type(x) is list:
            i = 0
            for a in x:
                attribute_name=name.rstrip('.') + '[' + str(i) + '].'
                if type(a) is dict or type(a) is list:
                    flatten(a, attribute_name)
                else:
                    out[attribute_name.rstrip('.')] = a
                i += 1
            # Create a new table for the list
            list_df = pd.DataFrame(x)
            list_df['row_number'] = name.split('.')[0]
            list_table = pa.Table.from_pandas(list_df)
            tables.append((name.rstrip('.'), list_table))
        else:
            out[name.rstrip('.')] = x

    flatten(y)
    return out, tables

def json_to_tables(json_data):
    # Spłaszczanie JSON
    flat_json, list_tables = flatten_json(json_data)

    # Konwersja do DataFrame
    df = pd.DataFrame([flat_json])

    # Dodanie numeru wiersza
    df['row_number'] = df.index + 1

    # Konwersja do PyArrow Table
    main_table = pa.Table.from_pandas(df)

    return main_table, list_tables


In [25]:
with open(paths_test[3], encoding='utf-8') as f:
    dt = json.load(f)
json_to_tables(dt)

C:\Users\tusiu\AppData\Local\Temp\ipykernel_17048\192641810.py:27: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  list_table = pa.Table.from_pandas(list_df)
C:\Users\tusiu\AppData\Local\Temp\ipykernel_17048\192641810.py:27: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  list_table = pa.Table.from_pandas(list_df)


(pyarrow.Table
 [0].title: string
 [0].year: int64
 [0].href: null
 [1].title: string
 [1].year: int64
 [1].genres[0]: string
 [1].href: string
 [1].extract: string
 [2].title: string
 [2].year: int64
 [2].genres[0]: string
 [2].genres[1]: string
 [2].genres[2]: string
 [2].href: string
 [2].extract: string
 [2].thumbnail: string
 [2].thumbnail_width: int64
 [2].thumbnail_height: int64
 row_number: int64
 ----
 [0].title: [["After Dark in Central Park"]]
 [0].year: [[1900]]
 [0].href: [1 nulls]
 [1].title: [["Clowns Spinning Hats"]]
 [1].year: [[1900]]
 [1].genres[0]: [["Silent"]]
 [1].href: [["Clowns_Spinning_Hats"]]
 [1].extract: [["Clowns Spinning Hats is a black-and-white silent film featuring clowns throwing hats back and forth to each other. It was written and produced by Lubin Films and released April 7, 1900."]]
 [2].title: [["Capture of Boer Battery by British"]]
 [2].year: [[1900]]
 ...,
 [('[0].cast',
   pyarrow.Table
   row_number: null
   __index_level_0__: null
   ----
  

In [26]:
import pandas as pd
import pyarrow as pa
import json

def flatten_json(y, name='', row_number=0):
    out = {}
    tables = []

    if type(y) is dict:
        for a in y:
            attribute_name= name + a +'.'
            out.update(flatten_json(y[a], attribute_name, row_number)[0])
            tables.extend(flatten_json(y[a], attribute_name, row_number)[1])
    elif type(y) is list:
        i = 0
        for a in y:
            attribute_name=name.rstrip('.') + '[' + str(i) + '].'
            if type(a) is dict or type(a) is list:
                out.update(flatten_json(a, attribute_name, i)[0])
                tables.extend(flatten_json(a, attribute_name, i)[1])
            else:
                out[attribute_name.rstrip('.')] = a
            i += 1
        # Create a new table for the list
        list_df = pd.DataFrame(y)
        list_df['row_number'] = row_number
        list_table = pa.Table.from_pandas(list_df)
        tables.append((name.rstrip('.'), list_table))
    else:
        out[name.rstrip('.')] = y

    return out, tables

def json_to_tables(json_data):
    # Spłaszczanie JSON
    flat_json, list_tables = flatten_json(json_data)

    # Konwersja do DataFrame
    df = pd.DataFrame([flat_json])

    # Dodanie numeru wiersza
    df['row_number'] = df.index + 1

    # Konwersja do PyArrow Table
    main_table = pa.Table.from_pandas(df)

    return main_table, list_tables


In [27]:
with open(paths_test[3], encoding='utf-8') as f:
    dt = json.load(f)
json_to_tables(dt)

C:\Users\tusiu\AppData\Local\Temp\ipykernel_17048\1926702172.py:27: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  list_table = pa.Table.from_pandas(list_df)
C:\Users\tusiu\AppData\Local\Temp\ipykernel_17048\1926702172.py:27: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  list_table = pa.Table.from_pandas(list_df)
C:\Users\tusiu\AppData\Local\Temp\ipykernel_17048\1926702172.py:27: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  list_table = pa.Table.from_pandas(list_df)
C:\Users\tusiu\AppData\Local\Temp\ipykernel_17048\1926702172.py:27: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  list_table = pa.Table.from_pandas(list_df)
C:\Users\tusiu\AppData\Local\Temp\ipykernel_17048\1926702172

(pyarrow.Table
 [0].title: string
 [0].year: int64
 [0].href: null
 [1].title: string
 [1].year: int64
 [1].genres[0]: string
 [1].href: string
 [1].extract: string
 [2].title: string
 [2].year: int64
 [2].genres[0]: string
 [2].genres[1]: string
 [2].genres[2]: string
 [2].href: string
 [2].extract: string
 [2].thumbnail: string
 [2].thumbnail_width: int64
 [2].thumbnail_height: int64
 row_number: int64
 ----
 [0].title: [["After Dark in Central Park"]]
 [0].year: [[1900]]
 [0].href: [1 nulls]
 [1].title: [["Clowns Spinning Hats"]]
 [1].year: [[1900]]
 [1].genres[0]: [["Silent"]]
 [1].href: [["Clowns_Spinning_Hats"]]
 [1].extract: [["Clowns Spinning Hats is a black-and-white silent film featuring clowns throwing hats back and forth to each other. It was written and produced by Lubin Films and released April 7, 1900."]]
 [2].title: [["Capture of Boer Battery by British"]]
 [2].year: [[1900]]
 ...,
 [('[0].cast',
   pyarrow.Table
   row_number: int64
   __index_level_0__: null
   ----
 